# Coleta e Classificação de Cursos ENAP

In [6]:
import re
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup
from unidecode import unidecode
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer, util
import string

# Baixa recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Função de limpeza básica
def limpar_competencia(texto: str) -> str:
    if pd.isna(texto):
        return ''
    texto = unidecode(texto.lower())  # minúscula + remove acentos
    texto = re.sub(r'[^a-z,\s]', '', texto)  # remove caracteres especiais
    return texto.strip()

# Remoção de pontuação
def remove_pontuacao(text):
    if pd.isna(text):
        return ""
    return "".join([i for i in text if i not in string.punctuation])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## PARTE 1 - Extração inicial de cursos da lista (CATÁLOGO)

In [4]:
url_base = "https://www.escolavirtual.gov.br/catalogo?page=1"
soup = BeautifulSoup(rq.get(url_base).text, "html.parser")

# Coleta total de páginas da paginação
paginas = [
    int(match.group(1))
    for a in soup.select("ul.pagination li a")
    if (match := re.search(r"page=(\d+)", a.get("href", "")))
]
ultima_pagina = max(paginas) if paginas else 1
print(f"Total de páginas encontrado: {ultima_pagina}")

# Estrutura de armazenamento inicial dos cursos
tabela_dic = {"ID": [], "Curso": [], "Status": [], "Link": [], "Conteudista": [], "Horas": []}

# Percorre todas as páginas dinamicamente
for pagina in range(1, ultima_pagina + 1):
    soup = BeautifulSoup(rq.get(f"https://www.escolavirtual.gov.br/catalogo?page={pagina}").text, "html.parser")

    for card in soup.select("div.card"):  # cada curso é um "card"
        a = card.select_one("h3.card-title a")
        if not a:
            continue

        curso = a.text.strip()
        link = a["href"]
        link_completo = link if link.startswith("http") else f"https://www.escolavirtual.gov.br{link}"
        curso_id = link_completo.rstrip("/").rsplit("/", 1)[-1]

        dd_tags = [dd.text.strip() for dd in card.find_all("dd") if dd.text.strip()]
        conteudista = dd_tags[0] if len(dd_tags) >= 1 else "N/A"
        horas = dd_tags[1] if len(dd_tags) >= 2 else "N/A"
        status = "Novo" if card.select_one("span.badge.text-bg-success") else "Normal"

        # Adiciona curso à tabela
        tabela_dic["Curso"].append(curso)
        tabela_dic["ID"].append(curso_id)
        tabela_dic["Link"].append(link_completo)
        tabela_dic["Conteudista"].append(conteudista)
        tabela_dic["Horas"].append(horas)
        tabela_dic["Status"].append(status)

# Salva CSV inicial
df = pd.DataFrame(tabela_dic)
df.to_csv("cursos_enap.csv", index=False, encoding="utf-8")


Total de páginas encontrado: 69


## Extração detalhada de cada curso

In [5]:
tabela_curso = {
    "Conteudo Programatico": [],
    "Descricao": [],
    "Certificador": [],
    "Disponibilidade": [],
    "Idioma": [],
    "Publico alvo": []
}

for linkCurso in tabela_dic["Link"]:
    soup = BeautifulSoup(rq.get(linkCurso).text, "html.parser")

    # Descrição (parágrafos)
    descricaoCurso = [p.text.strip() for p in soup.find_all("p", {"class": "fs-5"}) if p.text.strip()]
    descricao_texto = " ".join(descricaoCurso) if descricaoCurso else "N/A"
    tabela_curso['Descricao'].append(descricao_texto)

    # Conteúdo Programático
    modulos_raw = " ".join([dd.text.strip() for dd in soup.find_all("dd", {"class": "columns rich-text"})])
    tabela_curso["Conteudo Programatico"].append(modulos_raw if modulos_raw else "N/A")

    # Busca certificador, disponibilidade e idioma
    cert, disp, idioma = "N/A", "N/A", "N/A"
    infos = [div.text.strip() for div in soup.find_all("div", {"class": "flex-fill"}) if div.text.strip()]
    for txt in infos:
        if "Certificador:" in txt: cert = txt.replace("Certificador:", "").strip()
        elif "Disponibilidade:" in txt: disp = txt.replace("Disponibilidade:", "").strip()
        elif "Idioma:" in txt: idioma = txt.replace("Idioma:", "").strip()

    tabela_curso["Certificador"].append(cert)
    tabela_curso["Disponibilidade"].append(disp)
    tabela_curso["Idioma"].append(idioma)

    # Público-alvo
    publico_alvo = [div.text.strip() for div in soup.find_all("div", {"class": "flex-basis-100"}) if div.text.strip()]
    publico_final = publico_alvo[0].replace("Público-alvo:", "").strip() if publico_alvo else "N/A"
    tabela_curso["Publico alvo"].append(publico_final)

# Salva CSV detalhado
df = pd.DataFrame(tabela_curso)
df.to_csv("cursos_detalhado.csv", index=False, encoding="utf-8")

## Pré-processamento de textos

In [8]:
# Carrega CSVs salvos
cursos_enap = pd.read_csv("cursos_enap.csv")
cursos_detalhado = pd.read_csv("cursos_detalhado.csv")

# EXEMPLO de carregamento de competências (simulado)
competencias = pd.DataFrame({
    "Competências Vinculadas": [
        # Gestão e Administração
        "gestão de projetos", "gestão de processos", "gestão de riscos",
        "gestão de pessoas", "gestão pública", "planejamento estratégico",
        "administração pública", "orçamento público", "finanças públicas",

        # Liderança e Ética
        "liderança", "ética no serviço público", "tomada de decisão",
        "negociação", "resolução de conflitos", "trabalho em equipe",
        "comunicação assertiva", "gestão de equipes", "accountability",

        # Inovação e Tecnologia
        "inovação", "transformação digital", "governo digital",
        "inteligência artificial", "análise de dados", "big data",
        "segurança da informação", "blockchain", "computação em nuvem",

        # Direito e Legislação
        "direito administrativo", "direito constitucional",
        "licitações e contratos", "compliance", "transparência pública",
        "controle interno", "auditoria governamental",

        # Educação e Formação
        "andragogia", "metodologias ativas", "design instrucional",
        "avaliação de desempenho", "educação a distância",
        "formação continuada", "ensino híbrido",

        # Sustentabilidade e Sociedade
        "desenvolvimento sustentável", "agenda 2030", "responsabilidade social",
        "diversidade e inclusão", "acessibilidade", "governança ambiental",
        "políticas públicas", "gestão de cidades inteligentes",

        # Habilidades Transversais
        "criatividade", "pensamento crítico", "resolução de problemas",
        "organização pessoal", "gestão do tempo", "produtividade",
        "inteligência emocional", "adaptabilidade", "aprendizado contínuo",

        # Atendimento e Cidadania
        "atendimento ao cidadão", "gestão de serviços públicos",
        "satisfação do usuário", "ouvidoria", "participação social",
        "controle social", "accountability pública"
    ]
})


# Merge de datasets
cursos_detalhado["ID"] = cursos_enap["ID"].values
df_final = pd.merge(cursos_enap, cursos_detalhado, on="ID")
df_final = df_final[['ID', 'Curso', 'Descricao']]

# Limpeza textual
df_final['Descricao_limpa'] = df_final['Descricao'].apply(limpar_competencia)

df_final['Descricao_limpa'] = df_final['Descricao_limpa'].apply(remove_pontuacao).str.lower()

# Stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')
df_final['Descricao_limpa'] = df_final['Descricao_limpa'].apply(lambda x: [i for i in x.split() if i not in stopwords])

# Stemming
porter = PorterStemmer()
df_final['Descricao_limpa'] = df_final['Descricao_limpa'].apply(lambda x: [porter.stem(word) for word in x])

# Lemmatization
lemmatizer = WordNetLemmatizer()
df_final['Descricao_limpa'] = df_final['Descricao_limpa'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Texto final para embeddings
df_final['Descricao_para_modelo'] = df_final['Descricao_limpa'].apply(lambda x: " ".join(x))

## Embeddings e classificação

In [9]:
# Modelo de embeddings multilíngue
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Competências (normalizadas)
competencias_lista = competencias["Competências Vinculadas"].str.lower().unique().tolist()

# Geração de embeddings
descricao_embeddings = model.encode(df_final['Descricao_para_modelo'].tolist(), convert_to_tensor=True)
competencias_embeddings = model.encode(competencias_lista, convert_to_tensor=True)

# Similaridade coseno
similaridade = util.cos_sim(descricao_embeddings, competencias_embeddings)

# Definição de limiar
limiar = 0.5

# Atribuição multilabel de competências
df_final['Competencias provaveis'] = [
    [competencias_lista[i] for i, score in enumerate(sims) if score >= limiar]
    for sims in similaridade
]

# Exporta resultado final
df_final2 = df_final[['ID', 'Curso', 'Descricao', 'Competencias provaveis']]
df_final2.to_csv("correlacao_CursoXCompetencias.csv", index=False, encoding="utf-8")

print("✅ Pipeline finalizado! Arquivo correlacao_CursoXCompetencias.csv gerado.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Pipeline finalizado! Arquivo correlacao_CursoXCompetencias.csv gerado.
